# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='agent_env/Reacher')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# Solution

## Policy 

In [6]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Actor(nn.Module):
    
    def __init__(self, state_size, action_size, seed, fc1_units=256, fc2_units=256):
        """
        
        """
        super(Actor, self).__init__() 
        self.seed = torch.manual_seed(seed)
        self.model = nn.Sequential(
            #nn.BatchNorm1d(state_size),
            nn.Linear(state_size, fc1_units),
            nn.ReLU(),
            nn.Dropout(), 
            #nn.BatchNorm1d(fc1_units),
            nn.Linear(fc1_units, fc2_units),
            nn.ReLU(),
            nn.Dropout(), 
            #nn.BatchNorm1d(fc2_units),
            nn.Linear(fc2_units, action_size),
            nn.Tanh()
        ) 
        
        self.model.apply(self.init_weights)
        
    def init_weights(self, layer):
        if type(layer) == nn.Linear:
            nn.init.xavier_uniform_(layer.weight)
            layer.bias.data.fill_(0.1)
        
    def forward(self, state):
        """
        Build an actor (policy) network that maps states -> actions        
        """
        return self.model(state)              

## Agent

## Train 

In [7]:
import numpy as np
import random
import copy
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

In [3]:
LR = 1e-4         # learning rate
discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 10000
SGD_epoch = 4

In [10]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### Setup environment 

In [11]:
from unityagents import UnityEnvironment
import numpy as np

In [12]:
env = UnityEnvironment(file_name='agent_env/Reacher')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [13]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# get the state and action state of the environment 

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# size of each action
action_size = brain.vector_action_space_size

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]

print("action_size {}, state_size {}".format(action_size, state_size))

action_size 4, state_size 33


In [17]:
# Setup agent
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
policy = Policy(state_size=state_size, action_size=action_size, seed=2)

In [ ]:
# we use the adam optimizer (optim.SGD is also possible)
import torch.optim as optim
optimizer = optim.Adam(policy.parameters(), lr=LR)

In [4]:
# convert states to probability, passing through the policy
def states_to_prob(policy, states):
    states = torch.stack(states)
    policy_input = states.view(-1,*states.shape[-3:])
    return policy(policy_input).view(states.shape[:-3])

In [2]:
# collect trajectories for a parallelized parallelEnv object
def collect_trajectories(env, policy, brain_name, tmax=1000, nrand=5):    

    def generate_random_action(action_size=4):
        import random
        return [[random.uniform(-1.0, 1.0) for _ in range(action_size)]]
    
    def generate_zeroed_action(action_size=4):
        return [[0.0 for _ in range(action_size)]]
    
    #initialize returning lists and start the game!
    state_list = []
    reward_list = []
    action_list=[]

    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations
    
    # perform nrand random steps
    for _ in range(nrand):
        # ** take random action ** 
        env_info = env.step(generate_random_action())[brain_name]        
        # get next state 
        state = env_info.vector_observations
        # get reward 
        reward = env_info.rewards[0]
    
    for t in range(tmax):                
        state = torch.from_numpy(state).float().to(device)        
        
        with torch.no_grad():
            action = policy(state).cpu().data.numpy() 
            
        # clip action into valid range 
        action = np.clip(action, -1.0, 1.0)
        
        # take action 
        env_info = env.step(action)[brain_name]        
        # get next state 
        next_state = env_info.vector_observations
        # get reward 
        reward = env_info.rewards[0]
        # see if episode has finished 
        done = env_info.local_done[0]              
        
        # store the result
        state_list.append(state)
        reward_list.append(reward)
        action_list.append(action)
        done_list.append(done) 
        
        state = next_state
        
        # stop if we are done 
        if done:
            break

    # return states, actions, rewards
    return state_list, action_list, reward_list, done_list

In [5]:
def clipped_surrogate(policy, states, actions, rewards, discount=0.995, epsilon=0.1, beta=0.01):
    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    
    # convert everything into pytorch tensors and move to gpu if available
    actions = torch.tensor(actions, dtype=torch.int8, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)
    
    # clipped function
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)

In [ ]:
def train(n_episodes=2000):
    # keep track of progress
    mean_rewards = []
    
    for i_episode in range(1, n_episodes+1):
        # collect trajectories
        states, actions, rewards, dones = collect_trajectories(env, policy, brain_name, tmax)
        
        total_rewards = np.sum(rewards, axis=0)
        
        # gradient ascent step
        for _ in range(SGD_epoch):
            L = -clipped_surrogate(policy, states, actions, rewards, epsilon=epsilon, beta=beta)


In [18]:
def ddpg(n_episodes=2000, max_t=1000, print_every=100):
    
    scores_deque = deque(maxlen=print_every)
    scores = []
    max_score = -np.Inf
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations
        
        agent.reset() 
        score = 0 
        for t in range(max_t):
            agent.noise.reset()
            
            action = agent.act(state)
            
            env_info = env.step(action)[brain_name]        
            # get next state 
            next_state = env_info.vector_observations
            # get reward 
            reward = env_info.rewards[0]
            # see if episode has finished 
            done = env_info.local_done[0]  
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            
            if done:
                break 
                
        scores_deque.append(score)
        scores.append(score)
        max_score = max(max_score, score)                                
        
        if i_episode % print_every == 0:            
            print('\rEpisode {}\tAverage Score: {:.2f}\tMax score: {:.2f}'.format(i_episode, np.mean(scores_deque), max_score), end="")
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
                  
    return scores                        

In [17]:
def plot_scores(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
scores = ddpg()

Episode 300	Average Score: 0.03	Max score: 4.58

In [ ]:
plot_scores(scores)